# Homework 1. name, gender 읽어오기 - ETL 버전의 load 함수에 들어갈 내용

In [ ]:
import requests

def extract(url):
  f = requests.get(url)
  return f.text

In [ ]:
import csv

def transform(data):
  reader = csv.reader(data.strip().split('\n'))
  return reader

In [ ]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection():
  host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
  redshift_user = "ID"
  redshift_pass = "PW"
  port = 5439
  dbname = "dev"
  conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
      dbname=dbname,
      user=redshift_user,
      password=redshift_pass,
      host=host,
      port=port
  ))
  conn.set_session(autocommit=True)
  return conn.cursor()

"""
CREATE TABLE raw_data.name_gender (
  name varchar(64),
  gender varchar(16)
)
"""


## Homework 1-1. INSERT INTO SQL 완성하기

In [ ]:
def load(csv_reader):
  cur = get_Redshift_connection()
  # cur.execute(sql)를 사용
  for name, gender in csv_reader:
    print(name, gender)
    sql = "INSERT INTO raw_data.name_gender (name, gender) VALUES (%s, %s)"
    cur.execute(sql, [name, gender])

Extract 부터 함수를 하나씩 실행

In [ ]:
link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"

data = extract(link)
csv_reader = transform(data)
load(csv_reader)

## Homework 1-2. 만일 이 테이블의 업데이트 방식이 기존 테이블의 내용을 지우고 새로 업데이트 하는 방식이라면(Full-Refresh) 어떻게 하는 것이 좋을까?

In [ ]:
# 이건 잘 모르겠습니다. 최대한 유추해서 써봅니다.
def get_Redshift_connection():
  host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
  redshift_user = "ID"
  redshift_pass = "PW"
  port = 5439
  dbname = "dev"
  conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
      dbname=dbname,
      user=redshift_user,
      password=redshift_pass,
      host=host,
      port=port
  ))

  return conn


def load(csv_reader):
  conn = get_Redshift_connection()
  conn.autocommit = False

  # cur.execute(sql)를 사용
  for name, gender in csv_reader:
    print(name, gender)
    try:
      sql = "DELETE FROM raw_data.name_gender WHERE name = %s AND gender = %s"
      cur.execute(sql, [name, gender])
      sql = "INSERT INTO raw_data.name_gender (name, gender) VALUES (%s, %s)"
      cur.execute(sql, [name, gender])
      conn.commit()
    except:
      print("Failed to do transaction!")
      conn.rollback()

  if conn.is_connected():
    cursor.close()
    conn.close()
    

ETL 실행하기

In [ ]:
link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"

data = extract(link)
csv_reader = transform(data)
load(csv_reader)

<br><br>
<hr>

# Homework 2. Open Weathermap API : 서울 7일 평균 / 최소 / 최대 온도 읽어오기

API 연결해서 JSON data 가져오기

In [ ]:
import requests
from datetime import datetime

API_KEY = "your_api_key"

# latitude and longitude of Seoul, Korea
SEOUL_LAT = "37.5665"
SEOUL_LONG = "126.9780"

params = {
    "lat" : SEOUL_LAT,
    "lon" : SEOUL_LONG,
    "appid" : API_KEY
}

response = requests.get(url="https://api.openweathermap.org/data/2.5/onecall", params=params)
response.raise_for_status()
data = response.json()
daily = data['daily']

print(daily)

[{'dt': 1634439600, 'sunrise': 1634420499, 'sunset': 1634460779, 'moonrise': 1634456160, 'moonset': 1634406960, 'moon_phase': 0.39, 'temp': {'day': 282.23, 'min': 275.65, 'max': 283.2, 'night': 280.5, 'eve': 282.19, 'morn': 275.69}, 'feels_like': {'day': 280.2, 'night': 279.65, 'eve': 279.96, 'morn': 271.8}, 'pressure': 1028, 'humidity': 23, 'dew_point': 263.39, 'wind_speed': 6.53, 'wind_deg': 329, 'wind_gust': 11.6, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': 0, 'pop': 0, 'uvi': 4.01}, {'dt': 1634526000, 'sunrise': 1634506957, 'sunset': 1634547098, 'moonrise': 1634544120, 'moonset': 1634497200, 'moon_phase': 0.42, 'temp': {'day': 286.77, 'min': 279.45, 'max': 287.53, 'night': 285.36, 'eve': 286.1, 'morn': 279.45}, 'feels_like': {'day': 284.97, 'night': 284.46, 'eve': 285.04, 'morn': 279.45}, 'pressure': 1025, 'humidity': 30, 'dew_point': 269.59, 'wind_speed': 2.3, 'wind_deg': 240, 'wind_gust': 3.69, 'weather': [{'id': 500, 'main': 'R

서울 - 7일 (내일부터 일주일간) 평균 / 최소 / 최대 온도

In [ ]:
print("Date, Avg_temp, Min_temp, Max_temp")

for day in daily[1:]:
  date = datetime.fromtimestamp(day["dt"]).strftime('%Y-%m-%d')
  avg_temp = round(sum(day['temp'].values()) / len(day['temp']), 2)
  min_temp = day['temp']['min']
  max_temp = day['temp']['max']

  print(f"{date}, {avg_temp}, {min_temp}, {max_temp}")

Date, Avg_temp, Min_temp, Max_temp
2021-10-18, 284.11, 279.45, 287.53
2021-10-19, 284.13, 281.81, 287.01
2021-10-20, 283.32, 280.4, 286.48
2021-10-21, 284.77, 280.57, 288.33
2021-10-22, 285.88, 282.24, 289.28
2021-10-23, 286.74, 282.33, 290.58
2021-10-24, 287.95, 284, 291.34
